In [1]:
# read data that have annotations from zooniverse

import pandas as pd
import numpy as np
import os

dat = "../data/zooni/SUD Project Classifications_0105.csv" 
subject_dat = pd.read_csv("../data/zooni/sud-project-subjects.csv")
df = pd.read_csv(dat)

In [2]:
# filter out test annotations
df = df[df['workflow_name'] == 'Q1 - 16 annotators']

In [3]:
# get number of annotations per user
df.groupby('user_name').size().reset_index(name='counts')

,user_name,counts
0,Minh_Trinh,56
1,Rhadysh,301
2,SamihaZarin,15
3,Sushmitha9,305
4,aryanjain,303
5,chirayupatel,317
6,dwb65,3
7,geetakukreja,50
8,hassaan.alvi,119
9,laurenmiller324,276


In [4]:
# as per meeting 11/30 - filter annotations made by midniteclub
df = df[df['user_name'] != 'midniteclub']

In [5]:
# parse annotation from json
df['connection'] = df['annotations'].apply(lambda x: eval(x)[0]['value'])
df['subject'] = df['annotations'].apply(lambda x: eval(x)[1]['value'])
df['objective'] = df['annotations'].apply(lambda x: eval(x)[2]['value'])

In [6]:
# {"92915036":{"retired":null,"Filename":"text_2483.txt"}}
files = []
for line in df["subject_data"]:
     # Replace 'null' with 'None' and parse the line as a dictionary
    line = line.replace('null', 'None')
    line_dict = eval(line)
    # Access the first (and only) value in the dictionary
    first_value = list(line_dict.values())[0]
    # Extract the filename
    filename = first_value.get("Filename")
    if filename is not None:
        # Add the filename to the dataframe as a new column
       files.append(filename)

df['file'] = files

In [7]:
# get post text
subject_dat['file'] = subject_dat['metadata'].apply(lambda x: eval(x)['Filename'])
subject_dat['location'] = subject_dat['locations'].apply(lambda x: eval(x)['0'])

In [8]:
# filter 
subject_dat = subject_dat[subject_dat['subject_set_id'] == 116963]
subject_dat = subject_dat[['subject_id', 'file', 'location']]

In [ ]:
# read text data from URL
# import requests
# subject_dat['text'] = subject_dat['location'].apply(lambda x: requests.get(x).text) 

In [9]:
# how many posts have been annotated more than once
annotation_counts = df.groupby('file').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
print("number of posts annotated 1, 2, or 3 times: ", annotation_counts.groupby('counts').size())

number of posts annotated 1, 2, or 3 times:  counts
1    974
2    536
3    177
4     36
5      1
dtype: int64


In [10]:
print("num of annotations: ", len(df))
print("num of unique posts: ", len(df['file'].unique()))
print("num of posts that received at least 3 annotations: ", len(annotation_counts[annotation_counts['counts'] >= 3]))

num of annotations:  2726
num of unique posts:  1724
num of posts that received at least 3 annotations:  214


---
**For subset of posts with 2 annotations - re-merge them back in**

In [66]:
# as per meeting 11/30 - subset the annotations that have only 2 annotations
annotation_2counts = annotation_counts[annotation_counts['counts'] == 2]
# # unique files
annotation_2counts = annotation_2counts[['file']]
# #merge with post text
annotation_2counts = annotation_2counts.merge(subject_dat, on='file', how='left')

In [ ]:
# annotation_2counts.to_csv("../data/zooni/annotation_2counts_1204.csv", index=False)

In [11]:
# merge back in
df_2counts = pd.read_csv("../data/zooni/SUD Project Final Round_0104.csv")

In [12]:
files2=[]
for line in df_2counts["subject_data"]:
    # Replace 'null' with 'None' and parse the line as a dictionary
    line = line.replace('null', 'None')
    line_dict = eval(line)
    # Access the first (and only) value in the dictionary
    first_value = list(line_dict.values())[0]
    # Extract the filename
    filename = first_value.get("Filename")
    if filename is not None:
        # Add the filename to the dataframe as a new column
       files2.append(filename)
df_2counts['file'] = files2

In [13]:
print("num of annotations: ", len(df_2counts))
print("num of unique posts: ", len(df_2counts['file'].unique()))

num of annotations:  302
num of unique posts:  244


In [ ]:
# group by username count
# df_2counts.groupby('user_name').size().reset_index(name='counts')

In [14]:
import ast

# Assuming 'annotations' is a string representation of a list
df_2counts['annotations'] = df_2counts['annotations'].apply(ast.literal_eval)

In [15]:
def get_value(index, annotation_list):
    if len(annotation_list) > index:
        return annotation_list[index]['value']
    else:
        return None  # Adjust this according to your needs

df_2counts['connection'] = df_2counts['annotations'].apply(lambda x: get_value(0, x))
df_2counts['subject'] = df_2counts['annotations'].apply(lambda x: get_value(1, x))
df_2counts['objective'] = df_2counts['annotations'].apply(lambda x: get_value(2, x))

In [16]:
df_2counts = df_2counts[['user_name','subject_ids','file', 'connection', 'subject', 'objective']]

In [17]:
# get subjet data
subject_2counts = pd.read_csv("../data/zooni/SUD Project Final Round Subjects.csv")
subject_2counts = subject_2counts[subject_2counts['workflow_id'] == 25653]
subject_2counts['file'] = subject_2counts['metadata'].apply(lambda x: eval(x)['Filename'])
subject_2counts['location'] = subject_2counts['locations'].apply(lambda x: eval(x)['0'])

In [19]:
# get location and text
import requests

subject_2counts['text'] = subject_2counts['location'].apply(lambda x: requests.get(x).text)

In [18]:
# are filnenames unique
print("num of unique posts: ", len(subject_2counts['file'].unique()))

num of unique posts:  536


In [20]:
df_2counts = df_2counts.merge(subject_2counts[['file','text']], on='file', how='left')
# how many posts have been annotated more than once
df_2counts.groupby('text').size().reset_index(name='counts').sort_values(by='counts', ascending=False).groupby('counts').size()

counts
1    195
2     41
3      7
4      1
dtype: int64

In [ ]:
# import pickle
# pickle.dump(subject_2counts, open("../data/zooni/subject_2counts_0104.pkl", "wb"))

In [22]:
# add TEXT FOR DF
import pickle

subject_dat = pickle.load(open("../data/zooni/subject_dat.pkl", "rb"))

# left join to get text
df = df.merge(subject_dat, on='file', how='left')

In [32]:
# append with df
df = df[['user_name','subject_ids','file', 'connection', 'subject', 'objective', 'text']]
df_merged = pd.concat([df, df_2counts])

In [29]:
# how many with non nan values for connection_y count



0

In [30]:
# merge df2counts with df on text
# df_merge = df.merge(df_2counts[['text','connection','subject','objective']], on='text', how='left')

In [33]:
# BECAUSE THE SUBJECT IDS AND FILENAMES FOR THE SAME POST TEXT ARE DIFFERENT BETWEEN THE TWO DATASETS, I AM CREATING A NEW IDENTIFIER
# ANNOTATIONS OF THE SAME POST TEXT WILL HAVE THE SAME IDENTIFIER FOLLOWED BY THE ANNOTATION NUMBER ex. 1_1 is post 1, annotation 1

# group by text and assign new identifier
df_merged['id'] = df_merged.groupby('text').ngroup()
df_merged['annotation_num'] = df_merged.groupby('id').cumcount()+1
df_merged['combo_id'] = df_merged['id'].astype(str) + "_" + df_merged['annotation_num'].astype(str)

In [34]:
# subset to posts that have been annotated 3 & 4 & 5 times = 823 rows
df_merged.groupby('id').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
print("number of posts with at least 3 annotations: ", len(df_merged.groupby('id').size().reset_index(name='counts').sort_values(by='counts', ascending=False)[df_merged.groupby('id').size().reset_index(name='counts').sort_values(by='counts', ascending=False)['counts'] >= 3]))
# filter to posts that have been annotated >= 3
df_subset = df_merged[df_merged['id'].isin(df_merged.groupby('id').size().reset_index(name='counts').sort_values(by='counts', ascending=False)[df_merged.groupby('id').size().reset_index(name='counts').sort_values(by='counts', ascending=False)['counts'] >= 3]['id'])]

number of posts with at least 3 annotations:  221


In [35]:
# get stats for the annotated posts
print("num of unique posts: ", len(df_subset['id'].unique()))
print("num of unique annotations: ", len(df_subset['combo_id'].unique()))

num of unique posts:  221
num of unique annotations:  705


In [ ]:
pickle.dump(df_subset, open("../data/zooni/annotations_0104_complete.pkl", "wb"))
# pickle.dump(subject_dat, open("../data/zooni/subject_dat.pkl", "wb"))

---

### label selection

select the label to be used for the analysis for posts that have received all three annotations. selection is based on majority - meaning that if two or more annotators have selected the same label, this label will be used for the analysis. if no majority is found, the post will be excluded temporarily until agreement is reached.

In [36]:
# split df into 3 dfs
texts = df_subset[['id', 'text']].drop_duplicates()
connection = df_subset[['file', 'connection', 'text', 'id', 'combo_id']]
objective = df_subset[['file', 'objective', 'text', 'id', 'combo_id']]
subjective = df_subset[['file', 'subject', 'text', 'id', 'combo_id']]


In [37]:
# group by file and connection
connection2 = connection.groupby('id')['connection'].apply(list).reset_index()
# for each group, get the most common annotation, keep text
connection2['connection'] = connection2['connection'].apply(lambda x: max(set(x), key=x.count)) 
# add text
connection2 = connection2.merge(texts, on='id', how='left')


In [38]:
pickle.dump(connection2, open("../data/zooni/connection.pkl", "wb"))

In [39]:
subject2 = subjective.groupby('id')['subject'].apply(list).reset_index()
subject2['subject'] = subject2['subject'].apply(lambda x: max(set(x), key=x.count))
subject2 = subject2.merge(texts, on='id', how='left')
pickle.dump(subject2, open("../data/zooni/subject.pkl", "wb"))

In [40]:
# num unique posts
len(subjective['id'].unique())

221

In [41]:
# transpose and treat each class as a separate column
objectives = ['Quality', 'Legality', 'Effects', 'Methods of Ingestion', 'Combination of Substances', 'Mental Health',
          'N/A', 'Other', 'Overdose', 'Nurturant Support & Morality', 'Withdrawal', 'Safety', 'Relapse']

# create a column for each topic
obj = pd.DataFrame(columns=objectives)

objective['objective'] = objective['objective'].fillna('')  # Replace None with an empty string

for t in objectives:
    obj[t] = objective['objective'].apply(lambda x: 1 if t in x else 0)

obj = obj.fillna(0)

# add the text column
obj['Text'] = objective['text']
obj['id'] = objective['id']

/var/folders/jj/bt0ykz9n5_16z2gvbybl0tbc0000gn/T/ipykernel_6452/2911549128.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  objective['objective'] = objective['objective'].fillna('')  # Replace None with an empty string


In [42]:
# group by file and get the most common annotation for each class of objective
obj2 = obj.groupby('Text').agg(lambda x:x.value_counts().index[0])
obj2 = obj2.reset_index()
obj2.rename(columns={'Text': 'text'}, inplace=True)
# obj2 = obj2.merge(subject_dat, on='text')
pickle.dump(obj2, open("../data/zooni/objective.pkl", "wb"))

---

**CREATE TEST SET FOR BOB TEAM**

In [ ]:
# randomly select 100 posts
import random

random.seed(123)
test_set = random.sample(list(obj2['text']), 100)
test_set = pd.DataFrame(test_set, columns=['text'])

In [ ]:
# export
test_set.to_csv("../data/test_set.csv", index=False)

---

### Caluclate agreement for each label


In [ ]:
# fleiss kappa

# connection2['connection'].value_counts()
# subject2['subject'].value_counts()
# obj2['Quality'].value_counts()
# obj2['Legality'].value_counts()
# obj2['Effects'].value_counts()
# obj2['Methods of Ingestion'].value_counts()
# obj2['Combination of Substances'].value_counts()
# obj2['Mental Health'].value_counts()
# obj2['N/A'].value_counts()
# obj2['Other'].value_counts()
# obj2['Overdose'].value_counts()
# obj2['Nurturant Support & Morality'].value_counts()
# obj2['Withdrawal'].value_counts()
# obj2['Safety'].value_counts()
# obj2['Relapse'].value_counts()

# Fleiss' kappa
# from sklearn.metrics import cohen_kappa_score

# fk_connection = cohen_kappa_score(connection['connection'], connection['file'])
# fk_subject = cohen_kappa_score(subjective['subject'], subjective['file'])

# print("Fleiss' kappa for connection: ", fk_connection)
# print("Fleiss' kappa for subject: ", fk_subject)

In [43]:
import krippendorff as kd
from statsmodels.stats.inter_rater import fleiss_kappa, aggregate_raters

# subset dat
dat = connection[['file', 'connection']]
# transpose - raters as columns, subjects as rows

In [ ]:
# print first 5 rows as dict
# blah = dat.head().to_dict()
# blah = pd.DataFrame(blah)
# transposed_blah = blah.pivot_table(index='file', columns=df.groupby('file').cumcount() + 1, values='connection', aggfunc='first').reset_index()
# transposed_blah.columns = ['file', 'rater1', 'rater2', 'rater3']
# print(transposed_blah)

In [ ]:
# blah=connection[connection['id'] == 460]['connection'].value_counts()

In [44]:
# for posts that have 4,5 annotations, keep 3 that are most common
connection3 = {}
for idnum in connection['id']:
    subset = connection[connection['id'] == idnum]
    if len(subset) > 3:
        counts = subset['connection'].value_counts()
        # if the freq of one lable is greater than the other two combined, keep that label
        if len(counts) > 1 and counts[0] != counts[1]:
            label = counts.index[0]
            subset = subset[subset['connection'] == label]
            subset = subset.head(3)
            connection3[idnum] = subset 
        else:
            # if there is a tie, keep the first 3
            subset = subset.head(3)
            connection3[idnum] = subset    
    # else if the number of annotations is 3, keep all 3
    else:
        connection3[idnum] = subset
connection3 = pd.concat(connection3.values())

In [45]:
# get agreement
connection4 = connection3.pivot_table(index='id', columns=connection3.groupby('id').cumcount() + 1, values='connection', aggfunc='first').reset_index()
connection4.columns = ['file', 'rater1', 'rater2', 'rater3']


In [46]:
print(connection4.head())

   file       rater1       rater2       rater3
0     7   Disclosure   Disclosure   Disclosure
1    24  Inquisition  Inquisition  Inquisition
2    26   Disclosure  Inquisition   Disclosure
3    37  Inquisition   Disclosure   Disclosure
4    38   Disclosure   Disclosure   Disclosure


In [47]:
# Convert categorical values to integer codes
connection4[['rater1', 'rater2', 'rater3']] = connection4[['rater1', 'rater2', 'rater3']].apply(lambda x: x.astype('category').cat.codes)

# export
# connection3.to_csv("../data/zooni/connection_transpose_agreement.csv", index=False)

In [48]:
# get percent agreement
connection4['agreement'] = connection4.apply(lambda x: 1 if x['rater1'] == x['rater2'] == x['rater3'] else 0, axis=1)
print("Percent agreement: ", connection4['agreement'].sum()/len(connection4))


Percent agreement:  0.6787330316742082


In [49]:
reliability_data=connection4[['rater1', 'rater2', 'rater3']]
reliability_data2 = reliability_data.T

In [50]:
import krippendorff as kd
kappa = kd.alpha(reliability_data=reliability_data2, level_of_measurement='nominal')
print("Krippendorff's alpha for connection: ", kappa)

Krippendorff's alpha for connection:  0.5625360659704771


In [51]:
blah = subjective[subjective['id'] == 44]['subject'].value_counts()
blah.sort_values(ascending=False)
# keep 3 most common - whatever is in the first row plus the next if the total is not 3



Series([], Name: subject, dtype: int64)

In [52]:
subject3 = {}
for idnum in subjective['id']:
    subset = subjective[subjective['id'] == idnum]
    if len(subset) > 3:
        counts = subset['subject'].value_counts().sort_values(ascending=False)
        # 3 same and 1 or 2 different
        if counts[0] == 3 :
            label = counts.index[0]
            subset = subset[subset['subject'] == label]
            subject3[idnum] = subset 
        # 2 x 2 tie
        elif counts[0] == 2:
            subset = subset.head(3)
            subject3[idnum] = subset
        # if all 4 or 5 labels are the same, or none of the labels are the same
        else:
            subset = subset.head(3)
            subject3[idnum] = subset    
    # else if the number of annotations is 3, keep all 3
    else:
        subject3[idnum] = subset
subject3 = pd.concat(subject3.values())

In [59]:
subject4 = subject3.pivot_table(index='id', columns=subject3.groupby('id').cumcount() + 1, values='subject', aggfunc='first').reset_index()
subject4.columns = ['id', 'rater1', 'rater2', 'rater3']

In [65]:
# for evaluations, export subject4 with text
subject5 = subject4.merge(texts, on='id', how='left')
# filter for just 100% disagreements
subject5 = subject5[subject5['rater1'] != subject5['rater2']]
subject5.to_csv("../data/zooni/subject_transpose_agreement.csv", index=False)

In [54]:
# convert to categorical
subject4[['rater1', 'rater2', 'rater3']] = subject4[['rater1', 'rater2', 'rater3']].apply(lambda x: x.astype('category').cat.codes)

# export
# subject3.to_csv("../data/zooni/subject_transpose_agreement.csv", index=False)

In [55]:
subject4['agreement'] = subject4.apply(lambda x: 1 if x['rater1'] == x['rater2'] == x['rater3'] else 0, axis=1)
print("Percent agreement: ", subject4['agreement'].sum()/len(subject4))

Percent agreement:  0.29411764705882354


In [56]:
reliability_data=subject4[['rater1', 'rater2', 'rater3']]
reliability_data2 = reliability_data.T

kappa = kd.alpha(reliability_data=reliability_data2, level_of_measurement='nominal')
print("Krippendorff's alpha for subject: ", kappa)

Krippendorff's alpha for subject:  0.2113871693472884


In [ ]:
# objective
# AS OF 12/11, WE DECIDED NOT TO DO/REPORT AGREEMENT FOR OBJECTIVE
# objective3 = {}
# for file in obj['file']:
#     if len(obj[obj['file'] == file]) > 3:
#         objective3[file] = obj[objective['file'] == file].iloc[0:3]
#     else:
#         objective3[file] = obj[objective['file'] == file]

# objective3 = pd.concat(objective3.values())

In [ ]:
# reshape to 3d array
# arrays = []
# for objective in objectives:
#     arrays.append(objective3.pivot_table(index='file', columns=df.groupby('file').cumcount() + 1, values=objective, aggfunc='first').reset_index())
#     # rename
#     arrays[-1].columns = ['file', 'rater1', 'rater2', 'rater3']
#     # add agreement column
#     arrays[-1]['agreement'] = arrays[-1].apply(lambda x: 1 if x['rater1'] == x['rater2'] == x['rater3'] else 0, axis=1)


In [ ]:
# print percent agreement for each objective
# for i in range(len(arrays)):
#     print("Percent agreement for ", objectives[i], ": ", arrays[i]['agreement'].sum()/len(arrays[i]))
    

In [ ]:
# calculate kappa for each objective
# for i in range(len(arrays)):
#     reliability_data=arrays[i][['rater1', 'rater2', 'rater3']]
#     reliability_data2 = reliability_data.T
#     kappa = kd.alpha(reliability_data=reliability_data2, level_of_measurement='nominal')
#     print("Krippendorff's alpha for ", objectives[i], ": ", kappa)

---

**Annotation Inspection**

In [ ]:
# get distribution of annotation per student to assess if they are injecting noise
# how are the student annotating? are they annotating everything as 'other'?
df.groupby('user_name').size().reset_index(name='counts')
# get distribution of annotations per student
df.groupby('user_name')['connection'].value_counts().reset_index(name='counts')
# get percentages
# df.groupby('user_name')['connection'].value_counts(normalize=True)


In [ ]:
df.groupby('user_name')['subject'].value_counts().reset_index(name='counts')

In [ ]:
# plot distribution of subject annotations per student
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")
ax = sns.barplot(x="user_name", y="counts", hue="subject", data=df.groupby('user_name')['subject'].value_counts().reset_index(name='counts'))
plt.xticks(rotation=90)
plt.show()


In [ ]:
# what is the proportion of each class of subject annotation per student?
# as a percentage of all annotations
subject_by_student = df.groupby('user_name')['subject'].value_counts(normalize=True).reset_index(name='proportion')
subject_by_student['counts'] = df.groupby('user_name')['subject'].value_counts().reset_index(name='counts')['counts']
